In [14]:
#설치
#pip install google-cloud-speech
#pip install PyAudio-0.2.11-cp39-cp39-win_amd64.whl

In [8]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "speechtotext-345712-c93482c9ad85.json"

"""Google Cloud Speech API sample application using the streaming API.
NOTE: This module requires the additional dependency `pyaudio`. To install
using pip:
    pip install pyaudio
Example usage:
    python transcribe_streaming_mic.py
"""

import re
import sys
from google.cloud import speech
import pyaudio
from six.moves import queue
import numpy as np
from os import remove
import cv2
import keras

# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms
# tt파일 만들고 시작
li_num_zero = ['10']
with open('tt.txt','w',encoding='UTF-8') as f:
    for name in li_num_zero:
        f.write(name+'\n')

li = ['0']
with open('division_name.txt','w',encoding='UTF-8') as f:
    for name in li:
        f.write(name+'\n')

class MicrophoneStream(object):
    """Opens a recording stream as a generator yielding the audio chunks."""
    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk
        self._buff = queue.Queue()
        self.closed = True

    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            channels=1, rate=self._rate,
            input=True, frames_per_buffer=self._chunk,
            stream_callback=self._fill_buffer,
        )

        self.closed = False

        return self

    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        self._buff.put(None)
        self._audio_interface.terminate()

    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        """Continuously collect data from the audio stream, into the buffer."""
        self._buff.put(in_data)
        return None, pyaudio.paContinue

    def generator(self):
        while not self.closed:
            chunk = self._buff.get()
            if chunk is None:
                return
            data = [chunk]

            while True:
                try:
                    chunk = self._buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break
            yield b''.join(data)
def classification_number(img):
    load_model = keras.models.load_model('./num_class.h5')
    img = cv2.resize(img, (32,32), interpolation = cv2.INTER_AREA)
    pred = load_model.predict(img.reshape(1, 32, 32, 3))
    pred = list(pred[0])
    result = max(pred)
    if result > 0.9:
        return pred.index(result)
    else:
        return -1

def listen_print_loop(responses):
    num_chars_printed = 0
    cur_page = 0
    #저장되는 캡쳐개수
    count = 0
    #저장된 캡쳐개수에 따라 1씩증가
    num = 1
    #진행하는 전체 개수
    cheak = 1
    li = []
    #종이검출 후 인식되는 숫자개수
    num_count = 0
    while cv2.waitKey(33) < 0:
        for response in responses:
            result = response.results[0]
            transcript = result.alternatives[0].transcript
            overwrite_chars = ' ' * (num_chars_printed - len(transcript))

            if not result.is_final:
                sys.stdout.write(transcript + overwrite_chars + '\r')
                sys.stdout.flush()
                if  '다음' in transcript + overwrite_chars:
                    print('다음 설명서 출력')
                    cur_page += 1
                    with open('./tt.txt', 'w') as f:
                        f.write(str(cur_page))
                    
                if '이전' in transcript + overwrite_chars:
                    print('이전 설명서 출력')
                    cur_page -= 1
                    with open('./tt.txt', 'w') as f:
                        f.write(str(cur_page))
                        # 현재 화면 캡쳐jpg", frame)
                    # 저장되는 캡쳐 개수
                if '캡처' in transcript + overwrite_chars:
                    capture = cv2.VideoCapture(0)
                    ret, frame = capture.read()
                    cv2.imwrite('capture2{}'.format(count) + ".jpg",frame)
                    count += 1
                    capture.release()
                    
                    # 캡쳐된 이미지에 대해 이미지 사이즈 조절 + 분할 + 분할된 이미지 파일명 txt파일로 저장
                if '시작' in transcript + overwrite_chars:
                    # 저장된 캡쳐 개수와 분할하기위해 읽어온 개수가 같으면 stop
                    def reorderPts(pts): # 꼭지점 순서 정렬
                        idx = np.lexsort((pts[:, 1], pts[:, 0]))  # 칼럼0 -> 칼럼1 순으로 정렬한 인덱스를 반환
                        pts = pts[idx]  # x좌표로 정렬

                        if pts[0, 1] > pts[1, 1]:
                            pts[[0, 1]] = pts[[1, 0]] # 스와핑

                        if pts[2, 1] < pts[3, 1]:
                            pts[[2, 3]] = pts[[3, 2]] # 스와핑

                        return pts 
                    while num <= count:
                        # output 세로, 가로 크기
                        dw, dh = 1080, 1440
                        srcQuad = np.array([[0, 0], [0, 0], [0, 0], [0, 0]], np.float32)
                        dstQuad = np.array([[0, 0], [0, dh], [dw, dh], [dw, 0]], np.float32)
    #                     img = cv2.imread('fails.jpg')
                        img = cv2.imread('capture{}'.format(num) + ".jpg")
                        src_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                        _, src_bin = cv2.threshold(src_gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

                        # 외곽선 검출 및 명함 검출
                        contours, _ = cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

                        cpy = img.copy()
                        for pts in contours:
                            # 너무 작은 객체는 무시
                            if cv2.contourArea(pts) < 1000:
                                continue

                            # 외곽선 근사화
                            approx = cv2.approxPolyDP(pts, cv2.arcLength(pts, True)*0.02, True)

                            # 컨벡스가 아니고, 사각형이 아니면 무시
                            if not cv2.isContourConvex(approx) or len(approx) != 4:
                                continue

                            cv2.polylines(cpy, [approx], True, (0, 255, 0), 2, cv2.LINE_AA)
                            srcQuad = reorderPts(approx.reshape(4, 2).astype(np.float32))

                        pers = cv2.getPerspectiveTransform(srcQuad, dstQuad)
                        dst = cv2.warpPerspective(img, pers, (dw, dh))
                    # ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ검출개수 확인ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
                        imgray = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
                        rgbs = imgray.copy()

                        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
                        grad = cv2.morphologyEx(rgbs, cv2.MORPH_GRADIENT, kernel)
                        _, bw = cv2.threshold(grad, 0.0, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
                        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 1))
                        connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel)

                        contours, hierarchy = cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                        mask = np.zeros(bw.shape, dtype=np.uint8)

                        tmps = []
                        for idx in range(len(contours)):
                            x, y, w, h = cv2.boundingRect(contours[idx])
                            mask[y:y+h, x:x+w] = 0
                            cv2.drawContours(mask, contours, idx, (255, 255, 255), -1)
                            r = float(cv2.countNonZero(mask[y:y+h, x:x+w])) / (w * h)
                            if r > 0.45 and w > 8 and w < 50 and h > 8:
                                cv2.rectangle(rgbs, (x, y), (x+w-1, y+h-1), (0, 255, 0), 2)
                                tmps.append(dst[y:y+h-1, x:x+w-1,:])
                        # 50, 즉 흐릿한 이미지에대해 순서 이미지가 아니라고 판단하여 이진화 진행
                        for i in tmps:
                            ret, px = cv2.threshold(i, 50, 255, cv2.THRESH_BINARY)
                            # 다차원 배열을 for문을 통해 하나의 int값으로 받아와 0의개수 즉 
                            # 검정색에 해당하는 개수가 500개 이상이면 순서에 해당하는 번호라고 판단하여 count
                            num_c = 0
                            for i in px:
                                for j in i:
                                    for k in j:
                                        if int(k) == 0:
                                            num_c+=1
                            if num_c >= 300:
                                num_count += 1
                    # ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ분할ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
                        # 이미지 분할
                        imgray = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
                        img2=imgray.copy()
                        #이미지 이진화
                        blur = cv2.GaussianBlur(imgray, (3,3), 0) 
                        thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

                        # Morph operations
                        # 외곽선
                        edge = cv2.Canny(thresh, 100, 200) 
                        # count한 개수가 1개 이하면 세로축을 크게잡아 하나의 화면에 출력
                        if num_count <= 1:
                            kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(1000,400))
                            num_count = 0
                            print('이미지개수 = {}'.format(1))
                        # count한 개수가 2개 이상이면 세로축을 작게 잡아 여러개로 분할
                        elif num_count > 1:
                            kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(500,50))
                            num_count = 0
                            print('이미지개수 = {}'.format(2))
                        closed = cv2.morphologyEx(edge, cv2.MORPH_CLOSE, kernel) 

                        #contours가 찾은 경계의 배열 
                        contours, hierarchy = cv2.findContours(closed.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
                        contours_xy = np.array(contours , dtype=object) 
                        contours_xy.shape

                        #한페이지 내에서 이미지 순서대로 불러오기 
                        contours=reversed(contours)
                        #한페이지 내의 모든 폐곡선 범위에 대해 실행 
                        top=[]

                        for c in contours:
                            x,y,w,h = cv2.boundingRect(c)
                            top.append(y) 
                            total=len(top)-1
                            #동적 변수 생성으로 순서대로 이미지 입력
                            globals()['division_img{}'.format(cheak)] = imgray[y:y+h , x:x+w]
                            img = globals()['division_img{}'.format(cheak)]

                            if img.shape[0] > 350 and img.shape[1] > 1000:
                    #             cv2.imwrite('division_img{}'.format(cheak)+'.png', globals()['division_img{}'.format(cheak)])
                                img = img.reshape(img.shape[0], img.shape[1], 1)
                                rgb = img.copy()
                                kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
                                grad = cv2.morphologyEx(rgb, cv2.MORPH_GRADIENT, kernel)
                                _, bw = cv2.threshold(grad, 0.0, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
                                kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 1))
                                connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel)

                                contours, hierarchy = cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                                mask = np.zeros(bw.shape, dtype=np.uint8)

                                tmp = []
                                test = []
                                for idx in range(len(contours)):
                                    x, y, w, h = cv2.boundingRect(contours[idx])
                                    mask[y:y+h, x:x+w] = 0
                                    cv2.drawContours(mask, contours, idx, (255, 255, 255), -1)
                                    r = float(cv2.countNonZero(mask[y:y+h, x:x+w])) / (w * h)
                                    if r > 0.45 and w > 8 and w < 50 and h > 8:
                                        cv2.rectangle(rgb, (x, y), (x+w-1, y+h-1), (0, 255, 0), 2)
                                        # 실행하는 숫자의 크기가 50,50보다 작으므로 해당 조건을 통해 의미없는 이미지 제거
                                        test = img[y:y+h-1, x:x+w-1]
                                        if test.shape[0] <=50 and test.shape[1] <=50:
                                            tmp.append(img[y:y+h-1, x:x+w-1])
                                # 좌측 상단에 항상 순서 번호가 검출되기때문에 tmp[-1]에 대해 분류
                                tmp[-1] = cv2.cvtColor(tmp[-1], cv2.COLOR_GRAY2BGR)
                                print('예측한 값은 {}입니다.'.format(classification_number(tmp[-1])))
                                #모델이 인식한 숫자에따라 division_img?.png로 저장
                                cv2.imwrite('division_img{}'.format(classification_number(tmp[-1]))+'.jpg', globals()['division_img{}'.format(cheak)])
                                #li라는 리스트에 분할된 이미지 이름 저장
                                li.append('division_img{}'.format(classification_number(tmp[-1]))+'.jpg')
                            cheak += 1

                        num += 1
                    # division_name 라는 txt파일로 cam과 연계
                    with open('division_name.txt','w',encoding='UTF-8') as f:
                        for name in li:
                            f.write(name+'\n')
                    # tt.txt 숫자 0이 입력되며 시작
                    li_num_zero = ['0']
                    with open('tt.txt','w',encoding='UTF-8') as f:
                        for name in li_num_zero:
                            f.write(name+'\n')
                num_chars_printed = len(transcript)
            else:
                print(transcript + overwrite_chars)

                # Exit recognition if any of the transcribed phrases could be
                # one of our keywords.
                if re.search(r'\b(exit|quit)\b', transcript, re.I):
                    print('Exiting..')
                    break

                num_chars_printed = 0


def main():
    language_code = 'ko-KR'  # a BCP-47 language tag

    client = speech.SpeechClient()
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code=language_code)
    streaming_config = speech.StreamingRecognitionConfig(
        config=config,
        interim_results=True)

    with MicrophoneStream(RATE, CHUNK) as stream:
        audio_generator = stream.generator()
        requests = (speech.StreamingRecognizeRequest(audio_content=content)
                    for content in audio_generator)

        responses = client.streaming_recognize(streaming_config, requests)

        # Now, put the transcription responses to use.
        listen_print_loop(responses)
        

if __name__ == '__main__':
    main()
# [END speech_transcribe_streaming_mic]


캡처
 캡처
 캡처
 캡처
 캡처
이미지개수 = 2
예측한 값은 1입니다.
예측한 값은 2입니다.
이미지개수 = 1
예측한 값은 3입니다.
이미지개수 = 2
예측한 값은 4입니다.
예측한 값은 5입니다.
이미지개수 = 1
예측한 값은 6입니다.
이미지개수 = 1
예측한 값은 7입니다.


error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
